## El tipo de datos abstracto Cola

El Tipo Abstracto de Datos COLA es una **colección lineal** de objetos tal que la extracción de los objetos se realiza por el principio de la secuencia (*front*) y la inserción de elementos por su final (*rear*). 
Esta restricción implica que los objetos se insertan y se eliminan en una cola de acuerdo con el principio de primero en entrar, primero en salir (FIFO, *first in first out*). 

**_Aplicaciones de las Colas_**

En general, cualquier aplicación que implique el **acceso concurrente** a un recurso. Algunos ejemplos:
* En una impresora, el primer trabajo en llegar es el primero que se imprime (*first come first served*)
* Las peticiones que se  realizan a un servidor. Las peticiones se encolan a medida que se van recibiendo y se desencolan a medida que el servidor las procesa.
* **Planificador de los procesos a ejecutar por el sistema operativo**

<img src="./queue1.png"   width="450" height="450"  />

**_Interfaz del TAD COLA_**

El Tipo de Datos Abstracto COLA incluye los siguientes métodos para una cola **Q**:

   * **Q.enqueue($e$)**: Agrega el elemento $e$ al final de la cola $Q$.
   * **Q.dequeue( )**: Devuelve el elemento que ocupa el *front* de la cola, removiéndolo de la misma.  Ocurre un error si trata de extraerse un elemento de una cola vacía. 
   
Además, habitualmente, se definen también los siguientes métodos:

* **Q.first( )**: Devuelve un enlace al elemento que ocupa el *front* de la cola, sin removerlo; ocurre un error si la cola estuviese vacía
* **Q.back( )**: Devuelve un enlace al elemento que ocupa el *rear* de la cola, sin removerlo; ocurre un error si la cola estuviese vacía
* **Q.is_empty( )**: Devuelve `True` si la cola Q no contiene ningún elemento.
* **len(Q)**: Devuelve el número de elementos en la cola; en Python, lo implementaremos con el método especial `__len__()`.



Por convención, asumimos que **una cola recién creada está vacía y que no hay un límite *a priori* en la capacidad de la cola**. Los elementos agregados a la pila pueden tener un tipo arbitrario.

**_Ejemplo:_**

El siguiente ejemplo se instancia una cola `Q` de enteros y se realizan una serie de operaciones. En la tabla se muestra el retorno de cada una de las operaciones y el estado de la cola tras invocar al método correspondiente. En negrita se indica el elemento que ocupa el *front* de la cola.


|Operación | Retorno | Contenido de la pila |   
|---------:| -----   | ---               |
| Q = enqueue() | Q | [ ]|
| Q.enqueue(5) |–    | [5] |
| Q.enqueue(3) |– | [**5**, 3]
| len(s)   | 2 | [**5**, 3]
| Q.dequeue( ) | 5 | [**3**]
| Q.is_empty ( ) | False | [**3**]
| Q.dequeue( ) | 5 | [ ]
| Q.is_empty( ) | True | [ ]
| Q.dequeue( ) | *error* | [ ]
| Q.enqueue(7) | – | [**7**]
| Q.enqueue(9) | – | [**7**, 9]
| Q.first( ) | 7 | [**7**, 9]
| Q.enqueue(4) | – | [**7**, 9, 4]
| len(s) | 3 |  [**7**, 9, 4]
| Q.dequeue( ) | 7 | [**9**, 4]
| Q.enqueue(6) | – | [**9**, 6]
| Q.enqueue(8) | – | [9, 6, 8]
| Q.dequeue( ) | 9 | [**6**, 8]


## Implementación del TAD COLA

Nos planteamos diferentes estructuras de datos para almacenar los objetos de la cola: 
* Lista de Python
* Array de memoria estática
* Array circular con memoria estática
* Array circular con memoria dinámica 

### Estructura de datos: Lista de Python

In [1]:
class Queue:
    ''' EdD Lista Muy mala implementacion'''
    
    def __init__(self):
        self._data = []   # Instanciamos una lista como EdD para almenarlos objetos de la cola
        self._size = 0
    
    def enqueue(self, elem):
        self._data.append(elem)
        self._size += 1
        
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
            
        elem = self._data.pop(0)
        self._size -= 1
        return elem
    
    def first(self):
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        return self._data[0]
    
    def is_empty(self):
        return self._size == 0
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        if self.is_empty():
            return '< >'
        
        str_ = '<'
        for i in range(self._size - 1):
            str_ += str(self._data[i]) + ','
         
        str_ += str(self._data[self._size - 1]) + ">"
        return str_
    
#---- Driver Program

if __name__ == '__main__':
    q = Queue()
    print(q)
    
    q.enqueue(1)
    q.enqueue(2)
    q.enqueue(3)
    q.enqueue(4)
    
    print(q)
    print('Length:', len(q))
    
    q.enqueue(q.dequeue())
    q.enqueue(q.dequeue())
    print(q)


< >
<1,2,3,4>
Length: 4
<3,4,1,2>


### Estructura de datos: array memoria estática

**array:**
<img src="./fig_65_array.jpg"   width="550" height="550"  />

* Coste $O(1)$ de las operaciones de las primitivas de la cola. Sin embargo, **gestión ineficiente de la memoria**: Una secuencia de llamadas a *Q.dequeue()* y *Q.enqueue()* puede llevar a la cola a un estado en que se encuentre  llena cuando, en realidad, el array se hallase vacío.

In [2]:
class Queue:
    ''' EdD un array estático. Gestiona muy mal el espacio'''
    
    DEFAULT_CAPACITY = 4
    
    def __init__(self):
        self._data = [None] * Queue.DEFAULT_CAPACITY
        self._size = 0
        self._front = 0
    
    def enqueue(self, elem):
        if self._is_full():
            raise TypeError("The queue is full")
        
        index = self._front + self._size
        self._data[index] = elem 
        self._size += 1
        
    def dequeue(self):
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        elem = self._data[self._front]
        self._data[self._front] = None
        
        self._front += 1
        self._size -= 1
        
        return elem
    
    def first(self):
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        return self._data[self._front]
    
    def is_empty(self):
        return self._size == 0
    
    def _is_full(self):
        return self._front + self._size == Queue.DEFAULT_CAPACITY
    
    def __len__(self):
        return self._size
    
    def __str__(self):
        if self.is_empty():
            return '< >'
        
        str_ = '<'
        for i in range(self._size - 1):
            str_ += str(self._data[i]) + ','
         
        str_ += str(self._data[self._size - 1]) + ">"
        return str_
    
#---- Driver Program
if __name__ == '__main__':
    q = Queue()
    print(q)
    
    q.enqueue(1)
    q.enqueue(2)
    q.enqueue(3)

    print('Length:', len(q))
    print(q)
    print('First element:', q.first())

    # Secuencia de llamadas dequeue /enqueue()  
    q.enqueue(q.dequeue())
    q.enqueue(q.dequeue())

< >
Length: 3
<1,2,3>
First element: 1


TypeError: The queue is full

### Estructura de Datos: array circular, memoria estática

**array circular:**
<img src="./fig_66_circular.jpg"   width="550" height="550"  />

* Coste de las operaciones $O(1)$. Gestión eficiente de la memoria del array. Sin embargo, **la cola tiene un tamaño máximo:** El número máximo de objetos que se pueden almacenar en la cola está determinado por el tamaño inicial del array.

In [4]:
class Queue:
    '''FIFO Queue implementation using a static and circular array as underlying storage.'''
    DEFAULT_CAPACITY = 4
    
    def __init__(self):
        self._data = [None] * Queue.DEFAULT_CAPACITY  # nonpublic list instance
        self._size = 0
        self._front = 0  
    
    def enqueue(self, item):
        '''Add elemente to the back of queue
        
        Resize if the queue is full
        '''
        if self._is_full():
            raise TypeError('Queue is full')
        
        index = (self._front + self._size) % len(self._data)
        self._data[index] = item
        self._size += 1
    
    def dequeue(self):
        '''Remove and return the front element from the queue;
        
        Raise a TypeError exception if the queue is empty
        '''
        if self._size == 0:
            raise TypeError("Empty queue")
        
        e = self._data[self._front]
        self._data[self._front] = None    # help garbage collection
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        return e
    
    def _is_full(self):
        if self._size   == len(self._data):
            return True
        return False
    
    def is_empty(self):
        '''Return True if stack does not contain any elements.'''
        return self._size == 0
       
    def first(self):
        '''Return a reference to the front element of the queue, 
        
        Raise a TypeError exception if the queue is empty
        '''
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        return self._data[self._front]
        
    def __len__(self):
        '''Return the number of elements in the queue.'''
        return self._size
    
    def __str__(self):
        if self.is_empty():
            return '< >'
        
        str_ = '<'
        
        for i in range(self._size - 1):
            index = (self._front + i) % len(self._data)
            str_ += str(self._data[index]) + ','
         
        # add the last element to te output string 
        index = (self._front  +  self._size  - 1) % len(self._data)
        str_ += str(self._data[index]) + ">"
        return str_
    
#---- Driver Program

# NOTA: En este 'driver code' se accede explicitamente a la EdD de la cola.
# Esto NO debes hacerlo bajo ninguna circustancia. Aqui lo mostramos 
# únicamente por motivos docentes para mostrar la diferencia entre 
# el estdo de una cola q, accederemos a su contenido con la sntencia
# print(q), y el estado en ese mismo momento de la EdD con la que hemos
# implementado la cola y a a la que accedemos con print(q._data)

q = Queue()
print(q)

q.enqueue(1)
q.enqueue(2)
q.enqueue(3)

print('Length:', len(q))
print(q)
print(q._data)   # ATENCION: No acceder a variable privada
print('First queue element:', q.first())

#Secuencia de dequeue() / enqueue()
q.enqueue(q.dequeue())
q.enqueue(q.dequeue())
q.enqueue(q.dequeue())

print('Length:', len(q))
print(q)
print(q._data)

# El número máximo de elementos está determinado por 
# el tamaño del array
q.enqueue(4)
q.enqueue(5)
print(q)
print(q._data)

< >
Length: 3
<1,2,3>
[1, 2, 3, None]
First queue element: 1
Length: 3
<1,2,3>
[2, 3, None, 1]


TypeError: Queue is full

### Estructura de datos: array circular con memoria dinámica con crecimiento *geométrico*

**array circular:**

<img src="./fig_66_circular.jpg"   width="550" height="550"  />

**Resize geométrico:**
<img src="./fig_67_resize.jpg"   width="550" height="550"  />

* Permite que las primitivas de la cola tengan coste $O(1)$ y una gestión eficiente de la memoria.

In [6]:
class Queue:
    '''FIFO Queue implementation using a dynamic and circular array as underlying data structure.'''
    DEFAULT_CAPACITY = 4
    GROWTH_FACTOR = 2
    
    def __init__(self):
        self._data = [None] * Queue.DEFAULT_CAPACITY  # nonpublic list instance
        self._size = 0
        self._front = 0  
    
    def enqueue(self, item):
        '''Add elemente to the back of queue
        
        Resize if the queue is full 
        '''
        
        if self._is_full():
            self._resize(len(self._data) * Queue.GROWTH_FACTOR)
        
        index = (self._front + self._size) % len(self._data)
        self._data[index] = item
        self._size += 1
    
    def dequeue(self):
        '''Remove and return the front element from the queue;
        
        Raise a TypeError exception if the queue is empty
        '''  
        if self._size == 0:
            raise TypeError("Empty queue")
        
        e = self._data[self._front]
        self._data[self._front] = None    # help garbage collection
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        return e
    
    def _is_full(self):
        if self._size == len(self._data):
            return True
        return False
    
    def _resize(self, cap):
        '''Resize to a new list capacity'''
        old = self._data                  # keep track oth eold list
        self._data = [None] * cap         # new memorya alocation
        for k in range (0, self._size):   
            index = (self._front + k) % len(old)   # shift index
            self._data[k] = old[index]
            
        self._front = 0    
    
    def is_empty(self):
        '''Return True if stack does not contain any elements.'''
        return self._size == 0
       
    def first(self):
        '''Return a reference to the front element of the queue, 
        
        Raise TypeError exception if the queue is empty
        '''
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        return self._data[self._front]
        
    def __len__(self):
        '''Return the number of elements in the queue.'''
        return self._size
    
    def __str__(self):
        if self.is_empty():
            return '< >'
        
        str_ = '<'   # output string
        for i in range(self._size - 1):
            index = (self._front + i) % len(self._data)
            str_ += str(self._data[index]) + ','
         
        # add the last element to te output string 
        index = (self._front  +  self._size  - 1) % len(self._data)
        str_ += str(self._data[index]) + ">"
        return str_
    
#---- Driver Code

# NOTA: En este 'driver code' se accede explicitamente a la EdD de la cola.
# Esto NO debes hacerlo bajo ninguna circustancia. Aqui lo mostramos 
# únicamente por motivos docentes para mostrar la diferencia entre 
# el estdo de una cola q, accederemos a su contenido con la sntencia
# print(q), y el estado en ese mismo momento de la EdD con la que hemos
# implementado la cola y a a la que accedemos con print(q._data)

q = Queue()
print(q)

q.enqueue(1)
q.enqueue(2)
q.enqueue(3)

print('Length:', len(q))
print(q)
print(q._data)   # ATENCION: No acceder a variable privada
print('First queue element:', q.first())

q.enqueue(q.dequeue())
q.enqueue(q.dequeue())
q.enqueue(q.dequeue())

print('Length:', len(q))
print(q)
print(q._data)

q.enqueue(4)
q.enqueue(5)
print(q)
print(q._data)
len(q)


< >
Length: 3
<1,2,3>
[1, 2, 3, None]
First queue element: 1
Length: 3
<1,2,3>
[2, 3, None, 1]
<1,2,3,4,5>
[1, 2, 3, 4, 5, None, None, None]


5

## Análisis de la complejidad

Cuando se utiliza como **estructura de datos** para implementar el TAD Queue un **array con resize geométrico** 

|Operación | Compejidad temporal| Comentario| 
|---------| -----   |------ |
|Q.enqueue() | $O(1)^*$ | Coste amortizado con resize() |
|Q.dequeue() | $O(1)$ |  Coste amortizado con shrink() |
|Q.first() | $O(1)$| |
|Q.is_empty() | $O(1)$| |
|len(Q) | $O(1)$ | |

* Cuando el array es dinámico,  i.e., se utiliza *resize()* cuando la cola está llena, el coste $O(1)$ de la operación enqueue() es **coste amortizado** (ver el tema de pilas).
* En ciertas implementaciones para evitar el desperdicio de memoria que se produciría en una cola en el que número de sus elementos (size) fuese mucho menor que el tamaño del array, se incluye una estrategia de *shrinking (contracción)* realocando los elementos de la cola en un array de menor tamaño. **En este caso el coste $O(1)$ de la función _dequeue()_ también sería amortizado.** (ver p. 246 del libro de Goodrich et al.)
* La **complejidad espacial** (uso de espacio de memoria) para la cola **cuando no hay resize** es $O(n)$.

<div class="alert-warning">

**_Cuestiones_** 

* **(F)** Discuta las ventajas/inconvenientes de las siguientes estructuras de datos para almacenar los elementos de una cola. Justifique sus afirmaciones
 * Lista de Python
 * Array                     
 * Array circular con memoria estática
 * Array circular con memoria dinámica
 
* **(F)** Proporcione el código Python de los métodos `__init__()` y  `enqueue()` cuando se utilizan las estructuras de datos de la cuestión anterior.

* **(F)** Proporcione un método *_shirinking()* que *aloca* los objetos de la cola a un tamaño 1/2 de su capacidad máxima cuando el número de objetos en la cola es inferior a 1/3 a su capacidad máxima. ¿Cuando se invocaría a este método?

## Algoritmos

### Insertar un elemento en una cola ordenada

Proporcione el código Python de una función  `enqueue_in_order (Q:Queue, item)` que inserte un elemento en una *cola ordenada*, i.e., cuyos elementos están ordenados en sentido ascendente.
¿Cúal es el coste de la función? Discuta los resultados

In [7]:
def enqueue_in_order (Q:Queue, item):
    if Q.is_empty():
        Q.enqueue(item)
        return
    
    moves = 0
    while item > Q.first() and moves != len(Q):
        Q.enqueue(Q.dequeue())
        moves += 1
        
    Q.enqueue(item)
    
    for i in range(1, len(Q) - moves):
        Q.enqueue(Q.dequeue())
        
    return

#----- Driver program

if __name__ == '__main__':
    Q = Queue()

    enqueue_in_order(Q, 0)
    print(Q)

    # equeue a sequence of ordered elements 
    for i in range(5, 50, 5):
        Q.enqueue (i)
    print(Q)

    enqueue_in_order(Q, 26)
    print(Q)

<0>
<0,5,10,15,20,25,30,35,40,45>
<0,5,10,15,20,25,26,30,35,40,45>


### Utilizar una cola como EdD para implementar una pila 

* Describe how to implement the stack ADT using a single queue as an
instance variable, and only constant additional local memory within the
method bodies. What is the running time of the push(), pop(), and top()
methods for your design?
* Discute el coste de las operaciones de la Pila con esta implementación. 

In [10]:
class Empty(Exception):
    pass

class Stack:
    def __init__(self):
        self._data = Queue()
        
    def pop(self):
        return self._data.dequeue()
    
    def push(self, item):
        q_len = len(self._data)
        
        self._data.enqueue(item)
        
        # Empty Queue
        if q_len == 0:
            return
        
        # Reverse the elements queues
        for i in range(q_len):
            self._data.enqueue( self._data.dequeue())
            
    def is_empty(self):
        return self._data.is_empty()
    
    def top(self):
        if self._data.is_empty():
            raise Empty('Stack is empty')
        return self._data.front()
          
    def __str__(self):
        return str(self._data)
    
    def __len__(self):
        return len(self._data)
            
#-------Driver program

if __name__ == '__main__':
    S = Stack()

    # push elements in the stack
    for i in ['a','b','c']:
        S.push(i)
    print(f'Stack:{S}, size:{len(S)}')
      
    # pop elements from the stack
    while S.is_empty() == False:
        print(S.pop())
    print(f'Stack:{S}, size:{len(S)}')

    # check the stack top
    try:
        print(S.top())
    except Empty as z:
        print(z)

Stack:<c,b,a>, size:3
c
b
a
Stack:< >, size:0
Stack is empty


### TAD Deque (double ended queue)

**Referencia:** p. 247 del libro Goodrich et al.

Una cola doblemente terminada o *deque* (del inglés *double ended queue*) es una **colección de datos lineal que permite insertar y eliminar elementos por ambos extremos.** Podría verse como un mecanismo que permite aunar en una única estructura las funcionalidades de las pilas (estructuras LIFO) y las colas (estructuras FIFO), en otras palabras, estos tipos abstractos de datos (pilas y colas) podrían implementarse fácilmente con una deque. 

La interfaz del TAD Deque incluye las siguientes primitivas:

* **D.add_first($e$):** Añade el elemento $e$ al principio (*front*) del deque $D$ 

* **D.add_last($e$):** Añade el elemento $e$ al final (*rear*) del deque $D$ 

* **D.delete_first():** Elimina el primer elemento del deque $D$. Ocurre un error si el deque está vacío 

* **D.delete_last():** Elimina el último elemento del deque $D$. Ocurre un error si el deque está vacío 
* **D.is_empty():** DEvuelve True si el deque no contiene elementos

Adicionalmente se suelen incluir:
*  **D.first(e):** Devuelve el elemento situado al principio (*front*) del deque D **sin** extraerlo.
*  **D.last(e):** Devuelve el elemento situado al final (*rear*) del deque D **sin** extraerlo.
* **len(D):** Devuelve el número de elemntos en el deque


1. Proporciona una implementación del TAD Deque que utilice como **estructura de datos** un array circular con memoria dinámica.

**Nota:** En Python el operador $\%$ está implementado de forma tal que que la operación $-1 \% n = (n-1)$ siendo $n$ un entero mayor que cero. Esta implementación **no** es estándar para todos los lenguajes, por e.g. en ANSI C es diferente.  

In [14]:
class Deque:
    '''Dque implementation using a dynamic circular array as underlying data structure.'''
    DEFAULT_CAPACITY = 2
    GROWTH_FACTOR = 2
    
    def __init__(self):
        self._data = [None] * Deque.DEFAULT_CAPACITY  # nonpublic list instance
        self._size = 0
        self._front = 0  
    
    def add_last(self, item):
        '''Add elemente to the back of the deque
        
        Resize if the dequeu is full 
        '''
        if self._is_full():
            self._resize(len(self._data) * Deque.GROWTH_FACTOR)
        
        index = (self._front + self._size) % len(self._data)
        self._data[index] = item
        self._size += 1
    
    def add_first(self, item):
        '''Add elemente to the front of the deque
        
        Resize if the dequeu is full 
        '''
        if self._is_full():
            self._resize(len(self._data) * Deque.GROWTH_FACTOR)
        
        index = (self._front - 1) # equivalent (self._front - 1) % len(self._data)
        self._data[index] = item
        self._front = index
        self._size += 1
        
    def delete_first(self):
        '''Remove and return the element from the front (left) of the deque;
        
        Raise a TypeError exception if the the dequeu is empty
        '''
        
        if self._size == 0:
            raise TypeError("Empty deque")
        
        e = self._data[self._front]
        self._data[self._front] = None    # help garbage collection
        self._front = (self._front + 1) % len(self._data)
        self._size -= 1
        return e
    
    def delete_last(self):
        '''Remove and return the element from the rear (right) of the deque;
        
        Raise a TypeError exception if the the dequeu is empty
        '''
        
        if self._size == 0:
            raise TypeError("Empty deque")
        
        index = (self._front + self._size - 1) % len(self._data)     
        elem = self._data[index]
        self._data[index] = None
        self._size -= 1
        return elem
        
    def _is_full(self):
        if self._size == len(self._data):
            return True
        return False
    
    def _resize(self, cap):
        '''Resize to a new list capacity'''
        old = self._data                  # keep track oth eold list
        self._data = [None] * cap         # new memorya alocation
        for k in range (0, self._size):   
            index = (self._front + k) % len(old)   # shift index
            self._data[k] = old[index]     
        self._front = 0    
    
    def is_empty(self):
        '''Return True if stack does not contain any elements.'''
        return self._size == 0
       
    def first(self):
        '''Return a reference to the front element of the queue, 
        
        Raise TypeError exception if the queue is empty
        '''
        if self.is_empty():
            raise TypeError('Queue is empty')
            
        return self._data[self._front]
        
    def last(self):
        '''Return a reference to the front element of the queue, 
        
        Raise TypeError exception if the queue is empty
        '''
        if self.is_empty():
            raise TypeError('Queue is empty')
         
        index = (self._front + self._size - 1) % len(self._data)
        return self._data[index]    
    
    def __len__(self):
        '''Return the number of elements in the queue.'''
        return self._size
    
    def __str__(self):
        if self.is_empty():
            return '< >'
        
        str_ = '<'   # output string
        for i in range(self._size - 1):
            index = (self._front + i) % len(self._data)
            str_ += str(self._data[index]) + ','
         
        # add the last element to te output string 
        index = (self._front  +  self._size  - 1) % len(self._data)
        str_ += str(self._data[index]) + ">"
        return str_
    
#---- Driver Program

if __name__ == '__main__': 
    q = Deque()
    print(q)

    q.add_first(1)
    q.add_first(2)
    q.add_last(3)
    print(q)

    print('After delete last:', q.delete_last())
    print('Àfter delete first:', q.delete_first())
    print('After delete last:', q.delete_last())

< >
<2,1,3>
After delete last: 3
Àfter delete first: 2
After delete last: 1


### Deque from *collections*

Proporciona un *adptador* que implemente **nuestra especificación** del TAD Deque utilizando como estructura de datos el deque del paquete collections
```python
    from collections import deque
```

### Revertir una cola

Proporcione una función que revierta los elementos de una cola. 
```python
def queue_reverse(q:Queue):
    pass
```
No está permitido el uso de memoria auxiliar. Asuma que el TAD Queue tiene *solamente* los méodos *q.enqueue(), q.dequeu(), q.is_empty()* y *q.first()*. ¿Cúal es el coste de la función anterior? Justifique su respuesta.  

In [4]:
def queue_reverse(q):
    # Base case
    if q.is_empty():
        return
    
    # General case
    elem = q.dequeue()
    queue_reverse(q)
    q.enqueue(elem)
    
#--- Driver code

q = Queue()
for i in range(5):
    q.enqueue(i)
print(q)
queue_reverse(q)
print(q)



<0,1,2,3,4>
<4,3,2,1,0>
